In [1]:
import pandas as pd
import joblib
from fydjob.Database import Database

In [2]:
from fydjob.utils import lemmatize_words, remove_stopwords, join_strings

In [ ]:
db = Database()
df = db.to_frame()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/vlud/.pyenv/versions/3.8.6/envs/fydjob/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-b4a80be8d8cb>", line 2, in <module>
    df = db.to_frame()
  File "/home/vlud/code/OWNER_vlad/find-your-dream-job/fydjob/Database.py", line 151, in to_frame
    result = cur.execute(sql).fetchall()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/vlud/.pyenv/versions/3.8.6/envs/fydjob/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/vlud/.pyenv/versions/3.8.6/envs/fydjob/lib/py

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/vlud/.pyenv/versions/3.8.6/envs/fydjob/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-b4a80be8d8cb>", line 2, in <module>
    df = db.to_frame()
  File "/home/vlud/code/OWNER_vlad/find-your-dream-job/fydjob/Database.py", line 151, in to_frame
    result = cur.execute(sql).fetchall()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/vlud/.pyenv/versions/3.8.6/envs/fydjob/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/vlud/.pyenv/versions/3.8.6/envs/fydjob/lib/py

df = joblib.load('output/indeed_proc/processed_data.joblib')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['clean'] = df.job_text_tokenized.apply(join_strings).apply(lemmatize_words).apply(remove_stopwords)